In [1]:
import numpy as np
import matplotlib.pyplot as plt
from rasterio.transform import Affine
import rasterio as rio
import xarray as xr

In [3]:
## read data with xarray, extract values from nc file, and assign variables

data = xr.open_dataset('/home/tyler/cloud_advection/operational/data/test13.nc')

ghi = data['ghi'].values
ghi = np.asarray(ghi[0,:,:])

lats = data['lat'].values
lons = data['lon'].values

In [4]:
## calculate transform variable to create raster data

res = (lons[-1] - lons[0]) / 240.0
transform = Affine.translation(lons[0] - res / 2, lats[0] - res / 2) * Affine.scale(res, res)

In [5]:
# open in 'write' mode, unpack profile info to dst
## Create raster file

with rio.open(
   "test13.tif",
   "w",
   driver="GTiff",         # output file type
   height=ghi.shape[0],      # shape of array
   width=ghi.shape[1],
   count=1,                # number of bands
   dtype=ghi.dtype,          # output datatype
   crs="+proj=latlong",    # CRS
   transform=transform,    # location and resolution of upper left cell
) as dst:
   # check for number of bands
   if dst.count == 1:
       # write single band
       dst.write(ghi, 1)
   else:
       # write each band individually
       for band in range(len(ghi)):
           # write data, band # (starting from 1)
           dst.write(ghi[band], band + 1)